In [1]:
import sympy
import numpy as np
from tqdm import tqdm
import dill
dill.settings["recurse"] = True
from sympy import Rational as R 

In [2]:
(
    reference_x,
    reference_y,
    physical_x_0,
    physical_y_0,
    
    physical_x_1,
    physical_y_1,
    
    physical_x_2,
    physical_y_2,
    
    physical_x_3,
    physical_y_3,
    
    physical_x_4,
    physical_y_4,
    
    physical_x_5,
    physical_y_5,    
) = sympy.symbols("x y x_0 y_0 x_1 y_1 x_2 y_2 x_3 y_3 x_4 y_4 x_5 y_5")

f_interpolation = sympy.Matrix(sympy.symbols("f_0:6"))
s = sympy.symbols("s")

In [3]:
v0 = {reference_x: 0, reference_y: 0}
v1 = {reference_x: 1, reference_y: 0}
v2 = {reference_x: 0, reference_y: 1}

v3 = {reference_x: R(1, 2), reference_y: R(1, 2)}
v4 = {reference_x: 0, reference_y: R(1, 2)}
v5 = {reference_x: R(1, 2), reference_y: 0}

# Get basis functions

In [4]:
monomial_basis = sympy.Matrix(
[
 1,
 reference_x,
 reference_y,
 reference_x**2,
 reference_x*reference_y,
 reference_y**2
]
)

In [5]:
Vander = sympy.Matrix(np.zeros((6, 6)))

for i, basis in enumerate(monomial_basis):
        
    Vander[i, 0] = basis.subs(v0)
    Vander[i, 1] = basis.subs(v1)    
    Vander[i, 2] = basis.subs(v2)
    
    Vander[i, 3] = basis.subs(v3)
    Vander[i, 4] = basis.subs(v4)
    Vander[i, 5] = basis.subs(v5)    

In [6]:
lagrange_p2_basis = Vander.inv() @ monomial_basis

# Get mapping function

In [7]:
mapping_basis = lagrange_p2_basis

global_x = (
      mapping_basis[0] * physical_x_0
    + mapping_basis[1] * physical_x_1
    + mapping_basis[2] * physical_x_2    
    + mapping_basis[3] * physical_x_3
    + mapping_basis[4] * physical_x_4
    + mapping_basis[5] * physical_x_5
    
)
global_y = (
      mapping_basis[0] * physical_y_0
    + mapping_basis[1] * physical_y_1
    + mapping_basis[2] * physical_y_2    
    + mapping_basis[3] * physical_y_3
    + mapping_basis[4] * physical_y_4
    + mapping_basis[5] * physical_y_5
)

mapping_function = sympy.Matrix([global_x, global_y])

In [8]:
J_cofactor_T = mapping_function.jacobian([reference_x, reference_y]).cofactor_matrix().T
J = J_cofactor_T / J_cofactor_T.det()

In [9]:
u = sympy.Matrix([sympy.Function("u")(reference_x, reference_y)])
v = sympy.Matrix([sympy.Function("v")(reference_x, reference_y)])

In [10]:
grad_u = u.jacobian([reference_x, reference_y])@J_cofactor_T
grad_v = v.jacobian([reference_x, reference_y])@J_cofactor_T
weak_form = grad_u@grad_v.T

In [11]:
N = len(lagrange_p2_basis)

weak_form_functional = [[0 for i in range(N)] for j in range(N)]
weak_form_right_part = [[0 for i in range(N)] for j in range(N)]

for idx, jdx in tqdm([(idx, jdx) for idx in range(N) for jdx in range(N)]):
    first = lagrange_p2_basis[idx]
    second = lagrange_p2_basis[jdx]

    A = weak_form.subs({u[0]:first, v[0]:second}).doit()
    B = first * second

    weak_form_functional[idx][jdx] = A
    weak_form_right_part[idx][jdx] = B
        

weak_form_functional = sympy.Matrix(weak_form_functional) / J_cofactor_T.det()**2
weak_form_right_part = sympy.Matrix(weak_form_right_part)

weak_form_functional = weak_form_functional * abs(J_cofactor_T.det())
weak_form_right_part = weak_form_right_part * abs(J_cofactor_T.det())

100%|██████████| 36/36 [00:01<00:00, 19.40it/s]


In [12]:
weak_form_functional = sympy.Matrix(weak_form_functional)
weak_form_right_part = sympy.Matrix(weak_form_right_part)

f_interpolation = sympy.Matrix(f_interpolation)

In [13]:
lambdify_symbols = [
    reference_x,
    reference_y,
    
    physical_x_0,
    physical_y_0,
    
    physical_x_1,
    physical_y_1,
    
    physical_x_2,
    physical_y_2,
    
    physical_x_3,
    physical_y_3,
    
    physical_x_4,
    physical_y_4,
    
    physical_x_5,
    physical_y_5,
]

In [14]:
weak_form_functional_lambdified = sympy.lambdify(lambdify_symbols, weak_form_functional, cse=True)

In [15]:
weak_form_right_part = weak_form_right_part @ f_interpolation
weak_form_right_part_lambdified = sympy.lambdify([*lambdify_symbols, *f_interpolation], weak_form_right_part, cse=True)

In [16]:
quadrature_weights = np.array([
0.090817990382754,
0.036725957756467,
0.036725957756467,
0.036725957756467,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.072757916845420,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.028327242531057,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.009421666963733,
0.045321059435528,
0.045321059435528,
0.045321059435528,
]) / 2


quadrature_points = np.array([
[0.333333333333333,0.333333333333333],
[0.485577633383657,0.028844733232685],
[0.485577633383657,0.485577633383657],
[0.028844733232685,0.485577633383657],
[0.141707219414880,0.550352941820999],
[0.307939838764121,0.141707219414880],
[0.550352941820999,0.307939838764121],
[0.307939838764121,0.550352941820999],
[0.141707219414880,0.307939838764121],
[0.550352941820999,0.141707219414880],
[0.025003534762686,0.728323904597411],
[0.246672560639903,0.025003534762686],
[0.728323904597411,0.246672560639903],
[0.246672560639903,0.728323904597411],
[0.025003534762686,0.246672560639903],
[0.728323904597411,0.025003534762686],
[0.009540815400299,0.923655933587500],
[0.066803251012200,0.009540815400299],
[0.923655933587500,0.066803251012200],
[0.066803251012200,0.923655933587500],
[0.009540815400299,0.066803251012200],
[0.923655933587500,0.009540815400299],
[0.109481575485037,0.781036849029926],
[0.109481575485037,0.109481575485037],
[0.781036849029926,0.109481575485037],
])

In [17]:
def weak_form_functional_lambdified_quadrature(x_0, y_0, x_1, y_1, x_2, y_2, x_3, y_3, x_4, y_4, x_5, y_5):
    
    return weak_form_functional_lambdified(*quadrature_points.T, x_0, y_0, x_1, y_1, x_2, y_2, x_3, y_3, x_4, y_4, x_5, y_5)@quadrature_weights

In [18]:
def weak_form_right_part_lambdified_quadrature(x_0, y_0, x_1, y_1, x_2, y_2, x_3, y_3, x_4, y_4, x_5, y_5,
                                               f_0, f_1, f_2, f_3, f_4, f_5):
    
    return weak_form_right_part_lambdified(*quadrature_points.T, x_0, y_0, x_1, y_1, x_2, y_2, x_3, y_3, x_4, y_4, x_5, y_5,
                                           f_0, f_1, f_2, f_3, f_4, f_5)@quadrature_weights

In [19]:
interpolation_function = sympy.lambdify([*lambdify_symbols, *f_interpolation], (lagrange_p2_basis.T@f_interpolation)[0, 0], cse=True)

In [20]:
mapping_function_lambdified = sympy.lambdify(lambdify_symbols, mapping_function[:, 0], cse=True)

In [21]:
dill.dump(weak_form_functional_lambdified_quadrature, open("../calculations/lagrange_quadratic_weak_form_functional_p2", "wb"))
dill.dump(weak_form_right_part_lambdified_quadrature, open("../calculations/lagrange_quadratic_weak_form_right_part_p2", "wb"))
dill.dump(interpolation_function, open("../calculations/lagrange_quadratic_basis", "wb"))
dill.dump(mapping_function_lambdified, open("../calculations/lagrange_quadratic_mapping_function", "wb"))